In [1]:
import torch
from torch.autograd import Variable
import time

# Construct a Combined Loss

### Work with examples

In [2]:
output = torch.FloatTensor([[0.8, 0.2, 0.1, 5.8, 15.22],[0.4, 0.5, 0.1, 7.8, 20.22],[0.05, 0.45, 0.5, 5.8, 10.22]])
target = torch.FloatTensor([[1,0,0,0,0],[0,1,0,2.99,0],[0,0,1,0,20]])

In [52]:
p0 = target[:,0]
p1 = target[:,1]
p2 = target[:,2]
print(p0);print(p1);print(p2)

tensor([1., 0., 0.])
tensor([0., 1., 0.])
tensor([0., 0., 1.])


In [53]:
output[:,0]

tensor([0.8000, 0.4000, 0.0500])

In [54]:
torch.mul(output[:,0], p0)

tensor([0.8000, 0.0000, 0.0000])

In [55]:
logErr0 = torch.mul(torch.log(output[:,0]), p0)
logErr1 = torch.mul(torch.log(output[:,1]), p1)
logErr2 = torch.mul(torch.log(output[:,2]), p2)

In [56]:
mseErr1 = torch.mul(torch.pow(target[:,2+1] - output[:,2+1], 2), p1)
mseErr2 = torch.mul(torch.pow(target[:,2+2] - output[:,2+2], 2), p2)

In [57]:
logErr0 = -1 * torch.sum(logErr0)
logErr1 = -1 * torch.sum(logErr1)
logErr2 = -1 * torch.sum(logErr2)
mseErr1 = torch.sum(mseErr1)    
mseErr2 = torch.sum(mseErr2)    

In [58]:
def comb_error(output, target, sig1, sig2):
    output = (output + 0.0001)*0.999
    p0 = target[:,0]
    p1 = target[:,1]
    p2 = target[:,2]
    
    logErr0 = torch.mul(torch.log(output[:,0]), p0)
    logErr1 = torch.mul(torch.log(output[:,1]), p1)
    logErr2 = torch.mul(torch.log(output[:,2]), p2)
    mseErr1 = torch.mul(torch.pow(target[:,2+1] - output[:,2+1], 2), p1)
    mseErr2 = torch.mul(torch.pow(target[:,2+2] - output[:,2+2], 2), p2)   
    
    logErr0 = -1 * torch.sum(logErr0)
    logErr1 = -1 * torch.sum(logErr1)
    logErr2 = -1 * torch.sum(logErr2)
    mseErr1 = torch.sum(mseErr1)    
    mseErr2 = torch.sum(mseErr2)      
    
    return (1/output.shape[0]) * (logErr1 + logErr2 + (1/sig1)*mseErr1 + (1/sig2)*mseErr2)    

### MAE

In [60]:
output, target = output.numpy(), target.numpy()

y = target[:,2+1] + target[:,2+2]  
y

array([ 0.  ,  2.99, 20.  ], dtype=float32)

In [61]:
p1 = output[:,1]
p2 = output[:,2]
e1 = output[:,2+1]
e2 = output[:,2+2]
yhat = p1 * e1 + p2 * e2
yhat

array([2.6820002, 5.922    , 7.7200003], dtype=float32)

### Softmax

In [72]:
output1 = torch.FloatTensor([[8, 2, 1],[4, 5, 1],[5, 45, 50]])

In [73]:
torch.exp(output1)

tensor([[2.9810e+03, 7.3891e+00, 2.7183e+00],
        [5.4598e+01, 1.4841e+02, 2.7183e+00],
        [1.4841e+02, 3.4934e+19, 5.1847e+21]])

In [74]:
eo = torch.exp(output1)
sm = eo/torch.sum(eo, dim=1).view(-1,1)
sm

tensor([[9.9662e-01, 2.4704e-03, 9.0880e-04],
        [2.6539e-01, 7.2140e-01, 1.3213e-02],
        [2.8434e-20, 6.6929e-03, 9.9331e-01]])

In [75]:
torch.sum(sm, dim=1)

tensor([1.0000, 1.0000, 1.0000])

### Define error as function and apply autograd

In [38]:
def comb_error(output, target, sig1, sig2):
    output = (output + 0.0001)*0.999
    p0 = target[:,0]
    p1 = target[:,1]
    p2 = target[:,2]
    
    logErr0 = torch.mul(torch.log(output[:,0]), p0)
    logErr1 = torch.mul(torch.log(output[:,1]), p1)
    logErr2 = torch.mul(torch.log(output[:,2]), p2)
    mseErr1 = torch.mul(torch.pow(target[:,2+1] - output[:,2+1], 2), p1)
    mseErr2 = torch.mul(torch.pow(target[:,2+2] - output[:,2+2], 2), p2)   
    
    logErr0 = -1 * torch.sum(logErr0)
    logErr1 = -1 * torch.sum(logErr1)
    logErr2 = -1 * torch.sum(logErr2)
    mseErr1 = torch.sum(mseErr1)    
    mseErr2 = torch.sum(mseErr2)      
    
    return (1/output.shape[0]) * (logErr0 + logErr1 + logErr2 + (1/sig1)*mseErr1 + (1/sig2)*mseErr2)    

In [39]:
from torch.autograd import Variable

output = Variable(torch.FloatTensor([[0.4, 0.4, 0.2, 5.8, 15.22],[0.4, 0.5, 0.1, 7.8, 20.22],[0.05, 0.45, 0.5, 5.8, 10.22]]), requires_grad=True)
target = torch.FloatTensor([[1,0,0,0,0],[0,1,0,2.99,0],[0,0,1,0,20]])
err = comb_error(output, target, 100, 100)
err

tensor(1.1647, grad_fn=<MulBackward0>)

In [40]:
err.backward()

In [41]:
print(output.grad)

tensor([[-0.8331,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.6665,  0.0000,  0.0320,  0.0000],
        [ 0.0000,  0.0000, -0.6665,  0.0000, -0.0652]])


## Tests

In [42]:
def comb_error(output, target, sig1, sig2):
    output = (output + 0.0001)*0.999
    p0 = target[:,0]
    p1 = target[:,1]
    p2 = target[:,2]
    
    logErr0 = torch.mul(torch.log(output[:,0]), p0)
    logErr1 = torch.mul(torch.log(output[:,1]), p1)
    logErr2 = torch.mul(torch.log(output[:,2]), p2)
    mseErr1 = torch.mul(torch.pow(target[:,2+1] - output[:,2+1], 2), p1)
    mseErr2 = torch.mul(torch.pow(target[:,2+2] - output[:,2+2], 2), p2)   
    
    logErr0 = -1 * torch.sum(logErr0)
    logErr1 = -1 * torch.sum(logErr1)
    logErr2 = -1 * torch.sum(logErr2)
    mseErr1 = torch.sum(mseErr1)    
    mseErr2 = torch.sum(mseErr2)      
    
    return (1/output.shape[0]) * (logErr0 + logErr1 + logErr2 + (1/sig1)*mseErr1 + (1/sig2)*mseErr2)    

In [43]:
output = Variable(torch.FloatTensor([[0.4, 0.4, 0.2, 5.8, 15.22],[0.4, 0.5, 0.1, 7.8, 20.22],[0.05, 0.45, 0.5, 5.8, 10.22]]), requires_grad=True)
target = torch.FloatTensor([[1,0,0,0,0],[0,1,0,2.99,0],[0,0,1,0,20]])
err = comb_error(output, target, 100, 100)
err

tensor(1.1647, grad_fn=<MulBackward0>)

In [48]:
import numpy as np
repeats = 1000000
yh = np.array([[0.4, 0.4, 0.2, 5.8, 15.22],[0.4, 0.5, 0.1, 7.8, 20.22],[0.05, 0.45, 0.5, 5.8, 10.22]])
yh = np.repeat(yh, [repeats, repeats, repeats], axis=0)
y = np.array([[1,0,0,0,0],[0,1,0,2.99,0],[0,0,1,0,20]])
y = np.repeat(y, [repeats, repeats, repeats], axis=0)

In [49]:
output = Variable(torch.FloatTensor(yh), requires_grad=True)
target = torch.FloatTensor(y)
start = time.time()
err = comb_error(output, target, 100, 100)
end = time.time()
print(err)
print(end - start)

tensor(1.1647, grad_fn=<MulBackward0>)
0.8297433853149414
